In [1]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
writer = SummaryWriter()

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
df = pd.read_csv("/home/vaclavmatejka/devel/janestreet/model_8_SDAE_emphasized_loss/encoded.csv")
#df.loc[400:,"enc_feature_0":"enc_feature_49"]
#strip = 1 + df.shape[0] % 200
#df.loc[:df.shape[0]-strip]
#df["enc_feature_0", "feature_0"]
df

,Unnamed: 0,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,...,enc_feature_40,enc_feature_41,enc_feature_42,enc_feature_43,enc_feature_44,enc_feature_45,enc_feature_46,enc_feature_47,enc_feature_48,enc_feature_49
0,0,0.0,0.000000,0.009916,0.014079,0.008773,0.001390,0.006270,1.0,-1.872746,...,-0.123428,0.628796,-4.023776,-2.183965,-0.077025,-4.211199,1.769949,-1.008828,-0.767571,0.848031
1,1,0.0,16.673515,-0.002828,-0.003226,-0.007319,-0.011114,-0.009792,-1.0,-1.349537,...,-0.126286,0.982762,-0.533399,-0.803045,-1.049025,-1.367514,0.530526,-1.428573,0.224584,0.032676
2,2,0.0,0.000000,0.025134,0.027607,0.033406,0.034380,0.023970,-1.0,0.812780,...,-3.537209,0.427623,-0.817448,0.061715,-2.020541,-2.091881,-0.527815,0.721394,1.850761,3.646196
3,3,0.0,0.000000,-0.004730,-0.003273,-0.000461,-0.000476,-0.003200,-1.0,1.174378,...,-1.342461,1.185159,0.678673,1.851038,-2.856198,-0.807522,-0.994373,-0.755705,0.957583,2.259202
4,4,0.0,0.138531,0.001252,0.002165,-0.001215,-0.006219,-0.002604,1.0,-3.172026,...,0.006980,0.358059,-2.763735,-0.889712,0.208458,-2.821480,0.922953,-1.532228,-0.317601,0.308245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390395,3595,499.0,0.000000,0.000950,0.000950,-0.005755,-0.006558,-0.002785,1.0,-2.544545,...,2.371367,1.537548,1.859324,1.114790,2.582138,2.825742,-0.177744,1.865541,-2.093856,-2.508276
2390396,3596,499.0,0.501201,-0.000618,-0.000618,-0.015429,-0.077939,-0.085305,1.0,1.677197,...,0.737484,-0.531420,0.051283,-0.057722,0.434626,-0.072989,-1.063528,-0.836696,1.560309,-0.446988
2390397,3597,499.0,0.128814,0.008231,0.008231,0.012273,0.031198,0.031656,-1.0,1.727401,...,-0.164722,0.643131,0.073588,4.158679,-0.103347,1.852715,-0.158158,2.461866,-2.006335,1.108660
2390398,3598,499.0,13.542308,0.001497,0.001497,0.000088,-0.000681,0.001216,1.0,-0.746609,...,1.125191,2.172657,2.873001,0.482175,0.365660,1.764050,-1.104646,-0.787888,-1.091276,-3.244933


In [11]:
t = torch.tensor([1.0,2000.0,3.0])
noise = torch.randn_like(t)
noise.multiply(1)

tensor([-1.6380, -1.3042, -2.0674])

In [21]:
VALIDATION_SIZE = 1000
df = pd.read_csv("/home/vaclavmatejka/devel/janestreet/jane-street-market-prediction/train.csv", nrows=10000)


In [22]:
df = df.fillna(0)
df.loc[df['resp'] <= 0, 'trade'] = 0
df.loc[df['resp'] > 0, 'trade'] = 1

In [27]:
#df = df.sample(frac=1).reset_index()
df
df.loc[:,"date":"ts_id"]

,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,feature_2,...,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,ts_id
0,0,0.000000,0.009916,0.014079,0.008773,0.001390,0.006270,1,-1.872746,-2.191242,...,0.000000,1.168391,8.313583,1.782433,14.018213,2.653056,12.600292,2.301488,11.445807,0
1,0,16.673515,-0.002828,-0.003226,-0.007319,-0.011114,-0.009792,-1,-1.349537,-1.704709,...,0.000000,-1.178850,1.777472,-0.915458,2.831612,-1.417010,2.297459,-1.304614,1.898684,1
2,0,0.000000,0.025134,0.027607,0.033406,0.034380,0.023970,-1,0.812780,-0.256156,...,0.000000,6.115747,9.667908,5.542871,11.671595,7.281757,10.060014,6.638248,9.427299,2
3,0,0.000000,-0.004730,-0.003273,-0.000461,-0.000476,-0.003200,-1,1.174378,0.344640,...,0.000000,2.838853,0.499251,3.033732,1.513488,4.397532,1.266037,3.856384,1.013469,3
4,0,0.138531,0.001252,0.002165,-0.001215,-0.006219,-0.002604,1,-3.172026,-3.093182,...,0.000000,0.344850,4.101145,0.614252,6.623456,0.800129,5.233243,0.362636,3.926633,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1,0.420165,0.001847,0.016171,0.039813,0.061120,0.051909,-1,-0.803361,-1.687988,...,-2.735500,-0.603269,-1.588024,-0.614948,-2.783072,-0.882524,-1.890361,-0.789504,-1.662447,9995
9996,1,2.852259,-0.000712,-0.004864,-0.009795,-0.018601,-0.019537,1,-0.702446,-1.161989,...,-0.861067,-0.883083,-0.165111,-0.779975,-0.422061,-0.968161,0.167733,-0.920668,-0.010404,9996
9997,1,1.389255,0.002383,0.004190,0.002356,0.006833,0.009608,1,0.130875,-0.608811,...,0.276647,-0.001041,0.269809,-0.036464,0.415885,0.056424,0.422491,-0.036431,0.176114,9997
9998,1,0.085839,-0.003532,-0.037491,-0.053815,-0.052844,-0.065972,1,-3.172026,-3.093182,...,0.027756,0.940655,-0.102939,0.714865,-0.302707,1.022478,-0.315486,0.806448,-0.441524,9998


In [7]:
class JaneStreetDataset(Dataset):

    # Constructor with defult values
    def __init__(self, df, transform=None):
        self.df = df
        self.len = len(self.df)
        self.transform = transform

    # Getter
    def __getitem__(self, index):
        sample = torch.from_numpy(self.df.iloc[index]["feature_0":"feature_129"].values), torch.tensor(self.df.iloc[index]["resp"])
        sample = sample[0].to(device), sample[1].to(device)
        if self.transform:
            sample = self.transform(sample)
        return sample

    # Get Length
    def __len__(self):
        return self.len

In [8]:
df.iloc[2]["resp"]

-0.009634158690232266

In [9]:
df.iloc[2]["feature_0":"feature_129"].values

array([ 1.00000000e+00, -1.13518959e+00, -1.61827057e+00,  4.54450598e-01,
        2.80194598e-01,  8.34325776e-01,  6.00991096e-01, -7.52922825e-01,
       -7.60245730e-01, -1.26065357e+00, -1.16282365e+00,  3.28691328e-01,
        1.57604453e-01, -2.00281266e+00, -1.54143486e+00, -8.06410352e-01,
       -1.44557732e+00,  8.54325656e-01,  8.53438605e-01,  2.70115762e-01,
        1.22521804e-01,  1.07888779e+00,  4.84688120e-01,  2.30981202e-01,
        9.86927946e-02,  2.75536780e-01,  1.03283244e-01, -5.49189303e-01,
       -7.48393220e-01, -5.29760682e-01, -4.92395888e-01, -6.75317837e-01,
       -1.07180458e+00, -1.06146682e+00, -9.24106051e-01, -7.42370466e-01,
       -4.48241035e-01,  1.07317754e+00,  9.00970932e-01, -2.40498460e-01,
       -3.47299546e-01,  2.17191018e+00, -3.95957967e-01,  1.14263502e+00,
        4.47306184e-01,  5.67436716e-01,  5.70503068e-01,  7.82952089e-01,
        4.61510203e-01,  1.99919843e-01,  1.22670170e+00,  2.65245924e+00,
        1.34119908e+00, -

In [10]:
DROPOUT_P = 0.0
big_number = 1000
small_number = 500
model = torch.nn.Sequential(
    torch.nn.Linear(130, small_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(small_number),
    torch.nn.Linear(small_number, big_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(big_number),
    torch.nn.Linear(big_number, big_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(big_number),
    torch.nn.Linear(big_number, big_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(big_number),
    torch.nn.Linear(big_number, small_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(small_number),
    torch.nn.Linear(small_number, 1),
)
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

model.apply(init_weights)


<ipython-input-10-ae5219358239>:29: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


Sequential(
  (0): Linear(in_features=130, out_features=500, bias=True)
  (1): Dropout(p=0.0, inplace=False)
  (2): ReLU()
  (3): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): Linear(in_features=500, out_features=1000, bias=True)
  (5): Dropout(p=0.0, inplace=False)
  (6): ReLU()
  (7): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): Linear(in_features=1000, out_features=1000, bias=True)
  (9): Dropout(p=0.0, inplace=False)
  (10): ReLU()
  (11): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): Linear(in_features=1000, out_features=1000, bias=True)
  (13): Dropout(p=0.0, inplace=False)
  (14): ReLU()
  (15): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (16): Linear(in_features=1000, out_features=500, bias=True)
  (17): Dropout(p=0.0, inplace=False)
  (18): ReLU()
  (19): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, 

In [11]:
janestreet_train = JaneStreetDataset(df=df.iloc[:-VALIDATION_SIZE])
janestreet_validation = JaneStreetDataset(df=df.iloc[-VALIDATION_SIZE:])
BATCH_SIZE = 100
N_EPOCHS = 50
LR = 0.05
train_loader = torch.utils.data.DataLoader(dataset=janestreet_train, batch_size=BATCH_SIZE, drop_last=True)
validation_loader = torch.utils.data.DataLoader(dataset=janestreet_validation, batch_size=BATCH_SIZE, drop_last=True)

In [12]:
model = model.float()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.001)
criterion = nn.MSELoss()

In [13]:
def train_model(model, train_loader, validation_loader, optimizer, criterion, n_epochs=None):
    accuracy_list = []
    loss_list = []
    for epoch in range(n_epochs):
        for x, y in train_loader:
            model.train()
            optimizer.zero_grad()
            z = model(x.float())
            y = y.view(-1, 1).float()
            loss = criterion(z, y)
            writer.add_scalar("Train data", loss, epoch)
            loss.backward()
            optimizer.step()
            loss_list.append(loss.data)
        # perform a prediction on the validation data
        for x_test, y_test in validation_loader:
            model.eval()
            z = model(x_test.float())
            y = y.view(-1, 1)
            loss = criterion(z, y.float())
            writer.add_scalar("Validation data", loss, epoch)
            accuracy_list.append(loss.data)
    return accuracy_list, loss_list

In [14]:
accuracy_list, loss_list = train_model(model, train_loader, validation_loader, optimizer, criterion, n_epochs=N_EPOCHS)

KeyboardInterrupt: 

In [ ]:
model.eval()

In [ ]:
index=2000
#a = torch.from_numpy(df.iloc[index]["feature_0":"feature_129"].values)
a = df.iloc[index]["feature_0":"feature_129"].values
resp = torch.tensor(df.iloc[index]["resp"])

r = []
for one in range(BATCH_SIZE):
    r.append(a)
r = np.array(r)
model_input =torch.from_numpy(r).float().to(device)
z = model(model_input)

(resp, z[0])

In [ ]:
accuracy_list

In [ ]:
loss_list